# LLM Question Answering Tool

This notebook demonstrates how to build a tool that takes a technical question and generates an explanation using either an OpenAI GPT model or a locally hosted LLaMA model via Ollama. The project illustrates prompt processing, model integration, and streaming output to the notebook interface.

### Import Required Libraries

In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

### Define Model Names

In [2]:
# Models as constants

MODEL_GPT = 'gpt-4.1-mini'
MODEL_LLAMA = 'llama3.2'

### Load Environment Variables

In [3]:
# Set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

### Function to Stream Model Responses
This function detects whether to use an OpenAI GPT model or an Ollama LLaMA model and streams the model output directly to the notebook as markdown.

In [4]:
def stream_response(model, question):
    if "gpt" in model.lower():
        # Use OpenAI client for GPT models
        openai = OpenAI()
        stream = openai.chat.completions.create(
            model=model,
            messages=[{"role": "system", "content": question}],
            stream=True
        )

        response = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            response = response.replace("```","").replace("markdown", "")
            update_display(Markdown(response), display_id=display_handle.display_id)
    else:
        # For Ollama models, use requests directly
        response = ""
        display_handle = display(Markdown(""), display_id=True)

        stream = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": model,
                "prompt": question,
                "stream": True},
            stream=True
        )

        for line in stream.iter_lines():
            if line:
                json_line = json.loads(line.decode('utf-8'))
                if "response" in json_line:
                    token = json_line["response"]
                    response += token
                    update_display(Markdown(response), display_id=display_handle.display_id)

In [5]:
# Get gpt-4.1-mini to answer, with streaming
# Here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

stream_response(MODEL_GPT, question)

Certainly! Let's break down the code snippet:

python
yield from {book.get("author") for book in books if book.get("author")}


### What the code does:

1. **Set comprehension:**  
   python
   {book.get("author") for book in books if book.get("author")}
   
   This creates a **set** of authors extracted from the iterable `books`.

   - `books` is expected to be an iterable (like a list) of dictionaries (or dictionary-like objects).
   - For each `book` in `books`, it calls `book.get("author")`.
   - The `if book.get("author")` condition filters out any book where `"author"` is missing (returns `None` or something falsy).
   - The `.get("author")` ensures that if `"author"` is missing, it returns `None` instead of raising an error.
   - The result is a set of all unique authors found (because sets automatically eliminate duplicates).

2. **`yield from`**:  
   The statement:
   python
   yield from <iterable>
   
   is used inside a generator function to yield all elements from an iterable one by one.

   Here,
   python
   yield from { ... }
   
   will yield each unique author in the set to the caller, one at a time.

### Why this code is written this way:

- **Removing duplicates:**  
  Using a set comprehension removes duplicate authors automatically, so each author is yielded only once.

- **Filtering missing authors:**  
  Using the `if book.get("author")` filters out any books missing the `"author"` key or where it is `None` or falsy, so the generator only yields actual author values.

- **Use of `yield from`:**  
  Instead of looping and doing:
  python
  for author in { ... }:
      yield author
  
  the code uses `yield from` which is a concise way to yield every element of an iterable.

### Summary:

This code snippet is part of a generator function that yields **unique, non-empty authors** from a collection of book dictionaries, in no particular order, by:

- Gathering all authors into a set to ensure uniqueness and filter out missing values.
- Yielding each author one by one via `yield from`.

---

**Example:**

python
books = [
    {"title": "Book A", "author": "Alice"},
    {"title": "Book B", "author": "Bob"},
    {"title": "Book C", "author": "Alice"},
    {"title": "Book D"},  # author missing
    {"title": "Book E", "author": None}
]

def get_authors():
    yield from {book.get("author") for book in books if book.get("author")}

print(list(get_authors()))
# Possible output: ['Alice', 'Bob']  (order not guaranteed due to set)


This confirms the purpose and behavior of the original code.

In [6]:
# Get Llama 3.2 to answer
stream_response(MODEL_LLAMA, question)

This line of code uses a feature called generator expressions, which is a compact way to create iterators. Let's break it down:

1. `yield from`: This keyword is used with generator expressions to "forward" the iteration over another iterator.
2. `{book.get("author") for book in books if book.get("author")}`: This is an expression that generates values on-the-fly.

Here's what happens when this code is executed:

- It iterates over each item `book` in the collection of objects `books`.
- For each item, it checks if the dictionary inside `book` contains a key called `"author"` and has a value (i.e., not an empty string or None).
- If the condition is met, it yields the value of the `"author"` key within that dictionary.

However, there's something more going on here - `yield from` is used with `yield*`. The correct syntax should be:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

In this corrected version, the expression `(book.get("author")) for book in books if book.get("author")` would itself produce an iterator. However, `yield*` allows you to expand this into a larger iterable.

A better and more Pythonic way to write this code would be:

```python
for author in (book.get("author", None) for book in books if book.get("author")):
    yield author
```

Or using the list comprehension equivalent of generator expressions, as provided by `list` function:

```python
authors = [book.get("author") for book in books if book.get("author")]
for author in authors:
    yield author
```

This version does essentially the same thing but avoids using generators which are more suitable when you need to execute a piece of code once, then continue execution from where it left off. The generator expression is often used with `yield` or `yield*`, and the latter allows you to "forward" the iteration over another iterator.

The correct usage of `yield from` would be as follows:

```python
def get_authors():
    yield from {book.get("author") for book in books if book.get("author")}

# Usage:
for author in get_authors():
    print(author)
```

This version correctly uses the `yield from` expression.